<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Neural Signals and Signal Processing (NX-421)</h2>
<hr style="clear:both"></hr>

Welcome to the NX-421 class! In today's week, you will get to know Magnetic Resonance (MR) images.
You will learn today how to visualize images, along with some basic manipulation of MRI data!

In [ ]:
import fsl.wrappers
import mne_nirs
import nilearn
import mne
import mne_nirs
import dipy
import xml.etree.ElementTree as ET
import os.path as op
from fsleyes.views.orthopanel import OrthoPanel
import nibabel as nib
from nilearn.datasets import fetch_development_fmri
from fsl.wrappers import fslmaths
import glob
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import sys
sys.path.append('.')
import answer_creator

In [ ]:
# This will clear the view and remove our visualization totally:

def reset_overlays():
    """
    Clears view and completely remove visualization. All files opened in FSLeyes are closed.
    The view (along with any color map) is reset to the regular ortho panel.
    """
    l = frame.overlayList
    while(len(l)>0):
        del l[0]
    frame.removeViewPanel(frame.viewPanels[0])
    # Put back an ortho panel in our viz for future displays
    frame.addViewPanel(OrthoPanel)
    
def display_atlas_corresp(atlas_xml_path):
    """
    Given a path to an XML, extract all region labels and print their correspondance with numerical values
    of the nifti atlas file. (FSL-based convention)
    """
    root=ET.parse(atlas_xml_path).getroot()
    for type_tag in root.findall('data/label'):
        value = type_tag.get('index')
        display('{} : {}'.format(int(value)+1, type_tag.text))

This week, you will get a bit more familiar with visualization tools to view MRI images, diffusion MR images and lastly atlas visualization. We will introduce the different tools on the fly, so that you get a good idea of how to use them for your projects!

<h1><font color='black'>Part 1: Visualization and some basic manipulation </font></h1>


# 1. Visualization of MR images

These images are typically in a very peculiar format: the <a href="https://brainder.org/2012/09/23/the-nifti-file-format/">NIfTI file format</a>. You will notice that files have the .nii extension, or .nii.gz extension (which is nothing more than a compressed NIfTI file). While a full discourse on the format itself is a bit out of scope for this exercise, there are several things one should be aware of:
- A NIfTI file under the hood comprises two things, a header, and an image. The header keeps informations on the acquisition, such as dimensions of the voxels used, number of slices for each dimension, TR, affine transformation, sequence parameters...Some scanners (specifically SIEMENS scanners) will also store in the header information about the timing acquisitions of each slice, which is particularly handy when you need to perform slice-timing correction. The image itself is the data that has been acquired: the 3D volume in the case of a structural MR image!
- Because of its peculiar format and its 3D nature (for structurals again! a functional MRI would be 4D as it is a collection of 3D acquisitions), it is typically not possible to visualize them with usual image processing libraries such as ImageMagick: we need specialized software !

In this course, we will use <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki">FSL</a>.


<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b><img src="https://fsl.fmrib.ox.ac.uk/fsl/wiki_static/fsl/img/fsl-logo-x2.png">FSL</b></p>
<p style='text-indent: 10px;'>
FSL (FMRIB Software Library) is a library of analysis and visualization used for MRI, fMRI and DTI. It is typically run through command line. To make it easier on you, we are relying on the python interface of FSL, but you can also install the complete version on your system (it's free!) by following the instructions <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation">here</a>.<br>
    FSL is not the only library available! <a href="https://afni.nimh.nih.gov/">AFNI</a> and <a href="https://www.fil.ion.ucl.ac.uk/spm/">SPM</a> are two such other libraries, to quote two of the most well known. They all offer different flexibilities and ways to program. As an example, SPM is mainly reliant on MATLAB, while AFNI is heavy on the command-line. To serve as an easy hand-on introduction, we are using FSL, because Python is a language with which most of you are likely to be familiar at some level. Note that should you wish to use the command line commands of FSL, you can get quite detailed into the steps you wish to apply.
</p></span>
</div>

If you did launch this notebook using FSLeyes (if not, have a look at the README again!), you will now witness the unlimited power of the interactive session. For now you should have something empty, ie it might look like this:
<img src="imgs/fsleyes_empty.png">
<br>
### First visualization

We will add a first image in FSLeyes. This is done by specifying the absolute path to the image (in .nii or .nii.gz format !) 

In our case, we will start by showing a standard anatomical image: the brain in MNI space.
This brain is located in the path $FSLDIR/data/standard/MNI512_T1_0.5mm.nii.gz
Knowing this path, we can use FSLeyes load() function to directly load the image in the software interactively! 
Run the cell below:

In [ ]:
load(op.expandvars('$FSLDIR/data/standard/MNI152_T1_0.5mm'))

You should see something that looks like this:
<img src="imgs/mni_template_ex.png">

First of all, you should observe the specific contrast used: this is what is called a T1 contrast, typically used for anatomical scans. You will see later in the course why exactly it is called this way. For now, let's think about brain anatomy.
The ventricles appear **dark**. Why do you think that is? (Enter the answer number below)

1. The ventricles contain air, which appears dark in acquisition
2. The ventricles contain water, which appears dark in T1 contrast
3. The ventricles contain fat, which appears dark in T1 contrast

In [ ]:
answer = 0 # Replace here by 1, 2 or 3!
answer_creator.print_answer(answer)




Notice here we have picked a rather exquisite resolution: 0.5mm (meaning here a given voxel in 3D is 0.5 x 0.5 x 0.5 mm³)! 


Every time you move in a direction, you move across what are called **slices**. 


Can you make out the different types of tissues in the brain, just at a glance? Try to pinpoint where the majority of grey matter is found, based on contrast only.


Using your mouse, don't hesitate to play around the display a bit! Notice the 3-view display, showing in order sagittal, coronal and axial views of the brain. You can disable each view individually, by clicking on the appropriate buttons:
<img src="imgs/fsl_view_buttons.png"><br>


On the **sagittal slice**, you can see two letters at each side of the brain: P and A, for Posterior and Anterior, respectively. Moving only along the sagittal axis, can you approximately guess where the eye sockets are? (Note: you are allowed to move only along the sagittal axis, but you can still look at the other views, especially the coronal view!)

### Atlases and brain masks 

FSL has many functionalities. When looking at brain data, one interesting aspect is the one of atlases: figuring out which region one is looking at is useful to answer neurological questions or even to perform sanity checks on your data. As an example, if your participants are involved in a clicking task but no activity is visible in motor areas, you're probably doing something wrong somewhere. But to detect this, you obviously need to be able to distinguish **where** motor areas are located! 
One of its most basic uses is the ability to overlay images on top of each other. To demonstrate this one, we will load an atlas on top of our brain. 

<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>Atlases in MRI</b></p>
<p style='text-indent: 10px;'>
There are many atlases available, depending on your interest; some provide insight into subcortical structures, whereas others will partition the cortex. Atlases can be functional or anatomical, data-driven or labeled by experts, bilateral... All these choices will induce differences in the outlined regions, obviously. You should also be aware of the fact that atlases will provide you an image at the **population level** but will not be perfectly accurate at the population level.


In any case, an atlas is typically represented also a NIfTI file. Each region in the atlas - a region of interest (ROI) - takes a **unique** value to distinguish from the others. A companion file usually indicates the correspondance between the value and the region.
    
<img src="imgs/atlas_example.png">
    <center><i>One example of an atlas. Notice the colors matching in each hemisphere: this atlas does not differentiate between the two hemispheres for its regions.</i></center>
</p></span>
</div>


#### Loading an atlas and visualizing its regions

We will load our T1 (in 1mm resolution this time) and overlay an atlas image on top.
The colormap of the atlas can be set, which allows us to better visualize the different regions. In the right inferior corner of FSLeyes, you can view the value of the voxel at which the green cross is centered.
<img src="imgs/cross_value_fsleyes.png">


In [ ]:
reset_overlays()
load(op.expandvars('$FSLDIR/data/standard/MNI152_T1_1mm'))

In [ ]:
# Loads the atlas image
load(op.expandvars('$FSLDIR/data/atlases/MNI/MNI-maxprob-thr25-1mm.nii.gz'))
# Because there are two images in the list of overlays, the 2nd image being the atlas,
# we can very easily target its colormap and change it like so:
displayCtx.getOpts(overlayList[1]).cmap = 'Render3'

The companion file can be found in the FSL directory of the atlases, in xml format.

Here is the correspondance of values to regions in this atlas: 

In [ ]:
display_atlas_corresp(op.expandvars('$FSLDIR/data/atlases/MNI.xml'))

Using this equivalence, you can now locate and identify the different structures annotated by this atlas. Which sentence(s) is(are) accurate **for this atlas**?
- [ ] The thalamus is ventro-posterior of the caudate nucleus 
- [ ] The insula is a region anterior to the temporal lobe, lateral to the putamen and medial to the frontal cortex.
- [ ] The posterior precentral gyrus is part of the occipital lobe

#### Using atlases as masks

Now that you understand how atlases are handled, we can move on to another interest topic: masking data with an atlas. This can become handy for several reasons, the first being if you want to analyze data coming from a specific region. The mask can be created either in Python or through FSL. We will teach you both, so that you can choose whichever method you prefer. Let us create a mask of the frontal lobe, as an example.

##### Method 1: Python-based creation of the mask

The steps are simple:
1. Load the atlas as a regular file and extract the data as a numpy array
2. Create a binary mask of the frontal lobe
3. Write back to disk the mask with only the part of interest set to 1

Just to make sure you're still following and not falling asleep, we've provided you with a function that does the above steps...But you need to call it with the proper arguments to extract the frontal lobe *and* give a name to the file! By default, the mask will be named mymask.nii.

Lastly, to convince you that this mask is indeed the real deal, we will display it in FSLeyes!

In [ ]:
def make_mask_from_atlas(mask_value, mask_name="mymask.nii"):
    # Load the atlas with nibabel
    atlas = nib.load(op.expandvars('$FSLDIR/data/atlases/MNI/MNI-maxprob-thr25-1mm.nii.gz'))
    # Extract the atlas data as a numpy array
    atlas_data = atlas.get_fdata()
    # The mask is set to true only in voxels which match the target mask_value
    mask_data = atlas_data == mask_value
    # Create a new nifti image with nibabel
    mask_img = nib.Nifti1Image(mask_data.astype(np.uint8), atlas.affine, atlas.header)
    # Save the image
    nib.save(mask_img, mask_name)  

In [ ]:
mask_name=??? # Fill it with the name you want to give your mask!
mask_value=??? # Fill it with the value of the region you want to extract from the atlas!
make_mask_from_atlas(mask_value, mask_name)

reset_overlays()
load(op.expandvars('$FSLDIR/data/standard/MNI152_T1_1mm'))
load(mask_name)

##### Method 2: FSL-based creation of the mask

Again the steps are not too hard! You simply need to apply exactly the logic we have applied in method 1 to take the atlas, set all values of interest to 1 and the rest to zero. We will do so using a simple method: we will lower threshold the image, setting to zero anything **below** the provided threshold. Then, we will set anything **above** a second threshold to zero. Anything that is non zero by this point, we will set to 1.


We provide you with a simple function that accepts the two thresholds described above, named lower_threshold and upper_threshold. Decide on their value so as to extract again the *frontal lobe*. Likewise, the function will by default save the file as mymask.nii. Feel free to give it a more explicit name!

In [ ]:
def make_mask_from_atlas_fsl(lower_threshold, upper_threshold, mask_name="mymask.nii"):
    fslmaths(op.expandvars('$FSLDIR/data/atlases/MNI/MNI-maxprob-thr25-1mm.nii.gz')).thr(lower_threshold).uthr(upper_threshold).bin().run(mask_name)

In [ ]:
mask_name=???
lower_threshold=???
upper_threshold=???
make_mask_from_atlas_fsl(lower_threshold, upper_threshold, mask_name)

reset_overlays()
load(op.expandvars('$FSLDIR/data/standard/MNI152_T1_1mm'))
load(glob.glob(mask_name + '*')[0])

## 2. Viewing functional MR images

What you've learnt so far in visualization can very easily be applied to fMRI images. To showcase this, we will load one fMRI session. Based on what you have seen on MR images, and knowing that fMRI is a collection of MR images, what is true? (Multiple answers are possible)

1. An fMRI recording is in 4D because we add time
2. An fMRI recording is a collection of T1-weighted images
3. An fMRI recording is a collection of BOLD-weighted images
4. An fMRI can be both T1 or T2 weighted, depending on the TR and TE

In [ ]:
answers = [] # Put in this array all correct answers
answer_creator.print_fmri_properties_answer(answers)

In [ ]:
reset_overlays()
load(fetch_development_fmri(n_subjects=1, verbose=1, age_group='both')['func'][0])

Please pay attention to the fMRI's voxel dimensions:

In [ ]:
'Dimensions: ' + ' x '.join([str(i) + 'mm' for i in  nib.load(fetch_development_fmri(n_subjects=1, verbose=1, age_group='both')['func'][0]).header.get_zooms()])[:-2] + 's'

Do they match what you would expect? Compared to the anatomical you have seen so far, why do you think we observe such a difference in resolution?

Based on the fMRI you just opened, what can you say on the **contrast** of fMRI?
1. It is the same as the T1 MRI we saw: water is dark, fat is clear, neurons are dark, fibers are clear
2. It is the opposite of T1 MRI we saw: water is clear, fat is dark, neurons are clear, fibers are dark

In [ ]:
answer_mri = 0 # Replace with your answer
answer_creator.print_answer_contrast(answer_mri)

### fMRI as a movie

As you've no doubt noticed, FSLeyes has one little sneaky button: the movie button!
If you click on it, it will start to play the fMRI recording as a movie, allowing you to appreciate how the signal is fluctuating across time!


<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>🐞 Troubleshooting: fMRI movie is flickering 🐞</b></p>
<p style='text-indent: 10px;'>
    
If you enable the movie with FSLeyes synchronized with Python, you might not see a very pleasant movie, but more something along the lines of dark flickers with some occasional brains appearing in between.
<img src="imgs/buggy_fmri_display.gif">
    
This happens because FSLeyes is attempting to synchronize with Python and - well, there is a sluggish response between the two. Alleviating this behaviour is very easy! Simply disable the <b>Synchronize movie updates</b> option:
<img src="imgs/settings_button.png">
    <center><i>Click on the settings</i></center>
<img src="imgs/settings_menu.png">
    <center><i>Make sure to uncheck the Synchronize movie updates option</i></center>

</p></span>
</div>



## 3. Viewing diffusion MR images

Now that you are a bit familiar with FSLeyes, you should know you can also view other modalities, besides anatomical MR images with FSL; you can view functional data, but also diffusion weighted MR images!
We will show you how to view the fibers for an examplar dataset available in the DIPY library - a library specialized in diffusion imaging.

DIPY offers other visualizations that might interest you, so feel free to have a look in their documentation: https://dipy.org/tutorials/

In [ ]:
path_bundle = fetch_bundles_2_subjects()[1]

In [ ]:
files = glob.glob(op.join(path_bundle, "bundles_2_subjects/subj_1/bundles/*.trk")) 
reset_overlays()
load(op.join(path_bundle, "bundles_2_subjects/subj_1/t1_warped.nii.gz"))
[load(f) for f in files]

<h1><font color='black'>Part 2: Coregistration of images, a critical preprocessing step </font></h1>

Now that you're familiar with how fMRI works, we will have you conduct what is called coregistration.
As you will see in class, one of the key issues is to ensure MRI images are well aligned to each other. There are different reasons: comparison between participants during analysis, overlaying of anatomical MRI with functional MRI (to use for example an atlas based on anatomy), correction of motion...


For our purpose, we will align an anatomical MRI with the MNI anatomical template you displayed before.
This process is typically called **normalization**: we put the MRI into a "standard" space of reference. However the principle we'll show you can be applied to just any pair of images.
Start by visualizing the two!

The MRI and the template are not very well aligned, but we can try to make them more aligned. Specifically, we would like to find a transformation such that we can align our anatomical to the MNI template. This is the so-called normalization step.
So we need two ingredients to do this:
- A way to compute the transformation from anatomical to the MNI template (this step is called registration)
- A reference image in the space of the MNI template (here, actually, this is the MNI template)

#### What transformation?

What should this transformation be?
It is a combination of translation, rotation, scaling and other possible modifications, applied on our anatomical, so that it ends up matched to the target (the MNI image). In essence, the transformation fully describes the process to align the two images!


#### Why the reference ?
<p>
Let's pause for a moment. If the transformation already encapsulates all that there is to know about how to transform the volume, why do we need a reference from the target space?

To answer this, let's think about what happens when moving the anatomical image, shall we?
We rotate it, translate it, maybe shear it, cool, we have an anatomical well registered. But there's still an issue.
<br><b>What is the resolution of our anatomical?</b><br>
If you remember, the anatomical has an exquisite spatial resolution, but it might not be exactly the same as the MNI template: what if you used the MNI with 0.5mm voxel size from last week for example? In this case, we have a mismatch in resolutions. Let's get concrete with an example with Ducky!
</p>

#### Ducky's sunglasses
<p>    
Imagine the following example: we have an image of a duck (Ducky!), and we want to align sunglasses to it. 
Notice that this is hard, algorithmically, right? Unlike the brain, there  are no landmarks to use such as white-matter to try and optimize some cost. Sunglasses could be worn on the beak, the head, even the neck: there are no rules to fashion!
Fortunately, we are told Ducky should wear the glasses on its beak like the cool kids and are even provided the transformation to do so, a combination of translation, rotation and scaling. Applying this transform to the sunglasses, we get the following:
    
<div>
<center>
<div style="display:inline-block;">
    <img src="imgs/ducky/ducky_alone.png" style="width: 200px; height:auto; border: blue 6px groove;"/>
    <p style="text-align:center;">Ducky (our reference!)</p>
</div>
<div style="display:inline-block;">
    <img src="imgs/ducky/sunglasses_alone.png"/ style="width: 150px; height:auto;border: green 6px groove;" />
    <p style="text-align:center;">The sunglasses (in their own space)</p>
</div>   
<div style="display:inline-block;">
    <img src="imgs/ducky/ducky_summary.png" style="width: 200px; height:auto; border: blue 6px groove;"/>
    <p style="text-align:center;">What the transformation will do</p>
</div>
    </center>
</div>

Okay, we do all this. We obtain this:
<img src="imgs/ducky/ducky_total_uncropped.png" style="width: 200px; height:auto; border: black 6px groove;"/>    

Now, why is Ducky so big?
The answer is simple: the sunglasses and Ducky did not have the same:
- Resolution (meaning the glasses can get blurry, think of putting a 144px resolution screenshot on a 4K resolution image!)
- Field of view (canvas size if you're thinking of Photoshop layers for example)

Here are the field of view of the two images, kept as is :
<img src="imgs/ducky/ducky_uncropped.png" style="width: 200px; height:auto; border: black 6px groove;"/>    
    
But we would like to put the sunglasses on ducky without increasing the size around ducky. <b>So let's use Ducky's picture's resolution and field of view to correct our transformed sunglasses' own resolution and field of view</b>.
In other words, we will interpolate the sunglasses to match our duck's resolution and we will crop its field of view (or pad it if necessary) to match perfectly our duck. Final result:
    <center><img src="imgs/ducky/ducky_complete.png" style="width: 200px; height:auto; border: black 6px groove;"/>    <p style="text-align:center;">Much better.</p>
</center>

</p>

#### Back to neurological data

We use exactly the same idea when applying transformations. It is for this reason that when applying a transformation in FSL, you will always need to pass a reference image of the space in which you want to end up. This way, FSL will adapt the field of view but also the resolution by interpolation. This interpolation parameter can be done through nearest neighbour, trilinear or sinc interpolation.


## Types of normalization

So, you now know that you need a transformation and a reference. Great. Now, the transformation you allow can be of two types: it can be linear, meaning whatever you apply will be the same across the entire image, or non linear, where each voxel gets a separate treatment

(ducky linear and non linear)



### Linear normalization

To perform linear normalization, the idea is simple. The transformation we want should be linear - ie, affine.
Such a matching is usually called in image processing <a href="https://en.wikipedia.org/wiki/Image_registration">image registration</a>. Here, we're dealing with 3D data, so the problem is a bit more complicated. Fortunately all of this has been coded by very smart people, and to our rescue comes a tool specifically to register volumes to each other: <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT/UserGuide">FLIRT</a>!
<br>
This tool can allow many registrations and is extremely powerful. In its most basic form, it expects:
- An input volume, the volume you want to register (Ducky's sunglasses)
- A reference volume, to which the input is registered (Ducky's body)
- An output volume, the result of the transformation (Ducky's sunglasses once they are on Ducky's beak)

The situation looks like this:
(Img of the patient, img of the reference, img of the two overlaid )


Here is how you can call it to register the patient's anatomical to some reference sitting in another space (here the MNI152 template):
```python
flirt()

```

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'>
    FLIRT also expects the anatomical to be skull stripped to maximize normalization.</p>
</span>
</div>

We will first download some data. Do not worry about the exact command for now, it will become clear next week!

In [ ]:
import openneuro
from fsl.wrappers import bet
from mne.datasets import sample
import os

def mkdir_no_exist(path):
    if not op.isdir(path):
        os.makedirs(path)

def get_skull_stripped_anatomical(bids_root, preproc_root, subject_id):
    # We perform here skull stripping (you'll learn more about it next week!).
    # For now all you need to do is that we remove the bones and flesh from the MRI to get the brain!
    subject = 'sub-{}'.format(subject_id)
    anatomical_path = op.join(bids_root, subject, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
    betted_brain_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w'.format(subject_id))
    resulting_mask_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_mask'.format(subject_id))
    bet(anatomical_path, betted_brain_path, mask=resulting_mask_path)

def get_dataset_and_create_subfolders(dataset_id, bids_root, deriv_root, preproc_root, subject_id):
    mkdir_no_exist(bids_root)
    mkdir_no_exist(deriv_root)
    mkdir_no_exist(preproc_root)
    # Handy function to download datasets in a specific format! We will explain it next week!
    subject = 'sub-{}'.format(subject_id)
    os.system("openneuro-py download --dataset {} --include sub-{}/anat/* --target_dir {}".format(dataset_id, subject_id, bids_root))


dataset_id = 'ds004226'
subject_id = '001' 

sample_path = "dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

get_dataset_and_create_subfolders(dataset_id, bids_root, deriv_root, preproc_root, subject_id)
get_skull_stripped_anatomical(bids_root, preproc_root, subject_id)

In [ ]:
anatomical_target = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-001_T1w')
reference = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
result = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-{}_T1w_mni'.format(subject_id))
flirt(anatomical_target, reference, out=result)

Now, **whenever** we do a preprocessing step, we should inspect the result to assess if we did a proper job. This is called **quality control**!

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid #darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>💡 Quality Control (QC): the 1 - 10 - 100 dollar rule 💡</b></p>
<p style='text-indent: 10px;'>
The 1-10-100 rule states that it takes 1 dollar to verify and correct data at the start, 10 dollars to identify and clean data after the fact and 100 dollars to correct a failure due to bad data.

In preprocessing this is especially true. It will take you much less effort to first look at your data, detect what might be wrong from the start and deal with it rather than apply everything blindly and notice after the fact that something went wrong. Always, ALWAYS look at your data before <u>anything and any analysis</u>. A surgeon always looks at the patient before operating, you should do the same: you are surgeons to your dataset, so please look at it carefully (it's craving for attention, the poor thing 💔).
These intermediary steps of controlling the health of your dataset are called quality control steps. You're doing exactly what you might expect to do: you check the quality of your data before and after a given step, to ensure that nothing went awry for example.
<br>
</p></span>
</div>

Visualize the result of flirt on top of the reference. What do you think of alignment?

In [ ]:
reset_overlays()
load(reference)
load(result)

#### Choosing a cost
If you have a look at the options, you will notice that there is a cost option to flirt. Indeed, when performing registration, we have a function to measure how well the two images are matching one another. Flirt then attempts transformations to try and improve the fit. This function fit is defined through a cost, among different types.

Which cost should we use? If you were in a pure void, there would be no right or wrong answer from the get-go. No choice but to experiment and find out!

Hopefully, <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT/UserGuide#flirt">the documentation</a> should give you some pointers. What you want here is to register a T1 to a T1: this is a <u>within</u> modality registration, so you should restrict yourself only to costs appropriate to this type of modality! 

To help you, we've set up a cell that will run the different coregistrations for you. Simply fill in the different costs to consider :)

In [ ]:
possible_costs = ['mutualinfo', 'corratio', 'normcorr', 'normmi', 'leastsq', 'labeldiff']
costs_to_consider = [ ?? ] # fill me with the relevant costs

for c in costs_to_consider:
    flirt(anatomical_target, reference, out=result + '_' + c, cost=c)

And let's perform lastly the QC step for each of these nice costs. We'll leave it to you to decide if you prefer any cost!

In [ ]:
for c in costs_to_consider:
    load(result + '_' + c)

### Non linear normalization

So, you know how to do it linearly. Non linearly should be easy right?
<i>Well no, it's painfully hard</i>.
To do it, you can use <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FNIRT/">FNIRT</a>. You can browse through, the take-home message is that it is complicated, many steps are involved. 
Out of simplicity, we will not include it in this tutorial.

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'>
    FNIRT does NOT expect the input data to be skull-stripped.</p>
</span>
</div>

You've reached the end of this week's notebook! Congratulations!